In [1]:
pip install pandas numpy scikit-learn nltk streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
 import csv

fake = pd.read_csv('/content/Fake.csv',
                   quoting=csv.QUOTE_NONE,
                   encoding='utf-8',
                   on_bad_lines='skip',
                   engine='python')

real = pd.read_csv('/content/True.csv',
                   quoting=csv.QUOTE_NONE,
                   encoding='utf-8',
                   on_bad_lines='skip',
                   engine='python')



# Add label
fake['label'] = 0  # Fake
real['label'] = 1  # Real

# Combine and shuffle
df = pd.concat([fake, real], axis=0)
df = df.sample(frac=1).reset_index(drop=True)

# Use only title + text for now
df['content'] = df['title'] + " " + df['text']
df = df[['content', 'label']]

In [4]:

# Drop rows with missing content (if any)
df = df.dropna(subset=['content'])

# Ensure all content is string
df['content'] = df['content'].astype(str)

# Then apply text cleaning
df['content'] = df['content'].apply(clean_text)


/tmp/ipython-input-4-542927144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].astype(str)
/tmp/ipython-input-4-542927144.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].apply(clean_text)


In [5]:
'''def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text
'''
# Drop missing and convert to string
df = df.dropna(subset=['content'])
df['content'] = df['content'].astype(str)

# Apply cleaning
df['content'] = df['content'].apply(clean_text)


In [7]:
from sklearn.model_selection import train_test_split

X = df['content']       # Feature: cleaned news text
y = df['label']         # Target: 0 for fake, 1 for real

# Split into 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


Train size: 38
Test size: 13


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))  # includes bigrams
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [9]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

In [10]:

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score


model = PassiveAggressiveClassifier(max_iter=1000)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9230769230769231
Confusion Matrix:
 [[9 0]
 [1 3]]


In [11]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
